# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Шаг 1. Обзор данных

Откроем таблицу и изучим общую информацию о данных:
* Импортируем библиотеку `pandas` в сокращенном виде `pd`.
* Сохраним в переменную `data` файл `csv`, прочитав его при помощи библиотеки `pandas`.
* Для ознакомления с содержимым файла выведем его первые 15 строк на экран при помощи команды `display`.
* Применим к датафрейму метод `info` чтобы оценить количество пропусков в данных, тип значений в колонках и возможные ошибки в названии колонок датафрейма.

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
display(data.head(15))
data.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


# Шаг 2.1 Заполнение пропусков

В столбцах `'days_employed'` и `'total_income'` присутствуют пропуски которые нужно заполнить:
* Для начала посмотрим сколько всего пропусков в столбцах `'days_employed'` и `'total_income'`.
* Найдем какую долю составляют пропущенные значения в каждом из столбцов с пропусками, сохраним это отношение в переменную `'nan_in_data'` и выведем ее на экран.
* Заполним пропуски в столбце `'total_income'` на медианное значение этого столбца и проверим количество пропущенных значений.

In [2]:
print(data[['days_employed', 'total_income']].isna().sum())
print('')
nan_in_data = data.isna().sum()/len(data)*100
print('')
print(nan_in_data[['days_employed', 'total_income']])
print('')

total_income_median = data['total_income'].median()
print('Медианное значение столбца total_income:', total_income_median)

print('')
data['total_income'] = data['total_income'].fillna(total_income_median)
print(data[['total_income']].isna().sum())

days_employed    2174
total_income     2174
dtype: int64


days_employed    10.099884
total_income     10.099884
dtype: float64

Медианное значение столбца total_income: 145017.93753253992

total_income    0
dtype: int64


**Наблюдения:** в столбцах `'days_employed'` и `'total_income'` есть пропущенные значения типа `NaN` котороые составляют `10%` от всех значений в этих столбцах. Возможная причина появления пропусков это - отсутствие данных о стаже работы для пенсионеров. Замена пропусков в столбце менианым значением является лучшим решением потому что в столбце есть отдельные члены, которые намного больше или меньше остальных, а объём совокупности невелик.

# Шаг 2.2 Проверка данных на аномалии и исправления.

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Например, отрицательное количество дней трудового стажа в столбце `'days_employed'`. Для реальных данных это нормально. Обработаем значения в столбцах с аномалиями и опишем возможные причины появления таких данных. После обработки аномалий заполним пропуски в `'days_employed'` медианными значениями по этому столбцу:
* Для начала обратим все отрицательные значения в столбце `'days_employed'` на положительные. Создадим для этого функцию которая будет принимать все значения меньше `0` и умножать их на `-1`, а для всех остальных будет возвращать значения без изменений.
* После того как значения столбца преведены в положительные можно посчитать медианну и заполнить ей пропуски в столбце.
* Выведем первые 15 строк датафрейма на экран и проверим результат.
* Так же проверим количество пропусков в столбце функцией `.isna()` `.sum()`.

In [3]:
def plus_values(value):
    if value < 0:
        value *= -1
        return value
    else:
        return value
data['days_employed'] = data['days_employed'].apply(plus_values)

days_employed_median = data['days_employed'].median()
data['days_employed'] = data['days_employed'].fillna(days_employed_median)
display(data.head(15))
print('')
print(data[['days_employed']].isna().sum())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи



days_employed    0
dtype: int64


**Наблюдения:** отрицательные значения могли появиться в столбце из-за неправильной выгрузки данных или по причине неправильной работы счетчика. 

# Шаг 2.3. Изменение типов данных.

Заменим вещественный тип данных в столбце `'total_income'` на целочисленный:
* Для изменения значений в столбце применим функцию `.astype()` с арументом `'int'`.`
* Проверим результат, выведем первые 5 строк столбца `'total_income'` на экран.

In [4]:
data['total_income'] = data['total_income'].astype('int')
display(data['total_income'].head(5))

0    253875
1    112080
2    145885
3    267628
4    158616
Name: total_income, dtype: int64

# Шаг 2.4. Удаление дубликатов.

В данных присутствуют строки-дубликаты, удалим их. Также обработаем неявные дубликаты. Например, в столбце `'education'` есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к одному регистру. Проверим остальные столбцы.
* Для начала посмотрим сколько "явных" дубликатов есть во всем датафрейме, для этого применим функции `.duplicated()` и `.sum()`.
* Затем удалим все "явные" дубликаты из датафрейма и изменим индексацию при помощи функции `.drop_duplicates()` и `.reset_index()` с аргументом `drop=True` чтобы не создавать столбец со старыми значениями индексов.
* Проверим неявные дубликаты в столбцах которые могут содержать ошибки обусловленные человеческим фактором, для этого используем метод `.unique())`.
* Проверим формат столбца количества детей. Присутствует значение -1, а также 20 в столбце количества детей, -1 заменим на 1, а 20 заменим медианным значением, так как мы не знаем как возникла ошибка: случайно ввели лишний ноль при вводе 2 или же вместо 0 случайно поставили 2
* Проверим неявные дубликаты в столбце `'dob_years'` - возраст клиента в годах. Видим ошибку, присутствует значение 0. Заменим это значение на медианное для столбца.
* Приведем в единый формат столбец образование (уберем дубликаты, полученные из-за разного регистра букв).
* Проверим количество значений `XNA`в столбце `'gender'` чтобы проверить не помешают ли они в иследовании.
* Проведем проверку на "явные" дубликаты после приведения регистров в единый формат, затем удалим все дубликаты если такие будут обнаружены и востоновим индексацию.

In [5]:
print('Количество явных дубликатов:', data.duplicated().sum())
data = data.drop_duplicates().reset_index(drop=True)

print('')
print('Уникальные значения в столюце children:')
print(data['children'].unique())
print('')
print('Уникальные значения в столюце dob_years:')
print(data['dob_years'].unique())
print('')
print('Уникальные значения в столюце education:')
print( data['education'].unique())
print('')
print('Уникальные значения в столюце gender:')
print(data['gender'].unique())
print('')

data['children'] = data['children'].replace(-1, 1)
children_median = data.loc[data.loc[:, 'children'] != 20]['children'].median()
data['children'] = data['children'].replace(20, children_median)

dob_years_median = data.loc[data.loc[:, 'dob_years'] != 0]['dob_years'].median()
data['dob_years'] = data['dob_years'].replace(0, dob_years_median)

data['education'] = data['education'].str.lower()

print(data['gender'].value_counts())

print('')
print('Уникальные значения после удаления дубликатов столбце children:')
print(data['children'].unique())
print('')
print('Уникальные значения после удаления дубликатов столбце dob_years:')
print(data['dob_years'].unique())
print('')
print('Уникальные значения после удаления дубликатов столбце education:')
print( data['education'].unique())
print('')
print('Уникальные значения после удаления дубликатов столбце gender:')
print(data['gender'].unique())
print('')

print('Количество явных дубликатов после преведения регистров в единый формат:', data.duplicated().sum())
data = data.drop_duplicates().reset_index(drop=True)
print('Количество явных дубликатов после преведения регистров в единый формат:', data.duplicated().sum())

Количество явных дубликатов: 54

Уникальные значения в столюце children:
[ 1  0  3  2 -1  4 20  5]

Уникальные значения в столюце dob_years:
[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]

Уникальные значения в столюце education:
['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']

Уникальные значения в столюце gender:
['F' 'M' 'XNA']

F      14189
M       7281
XNA        1
Name: gender, dtype: int64

Уникальные значения после удаления дубликатов столбце children:
[1. 0. 3. 2. 4. 5.]

Уникальные значения после удаления дубликатов столбце dob_years:
[42. 36. 33. 32. 53. 27. 43. 50. 35. 41. 40. 65. 54. 56. 26. 48. 24. 21.
 57. 67. 28. 63. 62. 47. 34. 68. 25. 31. 30. 20. 49. 37. 45. 61. 64. 44.
 52. 

**Наблюдения:** в солбце `'gender'` 1 строка со значением пола — `XNA`. Скорее всего это пропуск, так как из данных не возможно сделать вывод о гендере заемщика оставим его как есть. Возможная причина появления дубликатов в данных это — человеческий фактор. 

# Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

* Создадим два новых датафрейма, в которых: каждому уникальному значению из `'education'` соответствует уникальное значение `'education_id'` — в первом; каждому уникальному значению из `'family_status'` соответствует уникальное значение `'family_status_id'` — во втором.
* Удалим дубликаты из словарей методом `.drop_duplicates()`
* Удалим из исходного датафрейма столбцы `'education'` и `'family_status'`, оставив только их идентификаторы: `'education_id'` и `'family_status_id'`.

In [6]:
education_dict = data[['education','education_id']]
family_status_dict = data[['family_status','family_status_id']]

education_dict = education_dict.drop_duplicates().reset_index(drop=True)
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
display(education_dict)
display(family_status_dict)        

data = data.drop(columns=['education', 'family_status'])
display(data.head())

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1.0,8437.673028,42.0,0,0,F,сотрудник,0,253875,покупка жилья
1,1.0,4024.803754,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0.0,5623.422610,33.0,1,0,M,сотрудник,0,145885,покупка жилья
3,3.0,4124.747207,32.0,1,0,M,сотрудник,0,267628,дополнительное образование
4,0.0,340266.072047,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу


# Шаг 2.6. Категоризация дохода.

На основании диапазонов, указанных ниже, создадим столбец `total_income_category` с категориями:
* 0–30000 — 'E';
* 30001–50000 — 'D';
* 50001–200000 — 'C';
* 200001–1000000 — 'B';
* 1000001 и выше — 'A'.
Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`.

In [7]:
# создадим функцию, которая категоризирует данные из столбца total_income
def total_income_category(row):
    total_income = row['total_income']
 
    if total_income <= 30000:
        return 'E'
    if total_income <= 50000:
        return 'D'
    if total_income <= 200000:
        return 'C'
    if total_income <= 1000000:
        return 'B'
    return 'A'
 
# сформируем новый столбец
data['total_income_category'] = data.apply(total_income_category, axis=1)
 
# выведем первые 10 строк для проверки
data.head()


,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1.0,8437.673028,42.0,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1.0,4024.803754,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0.0,5623.422610,33.0,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3.0,4124.747207,32.0,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0.0,340266.072047,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу,C


# Шаг 2.7. Категоризация целей кредита.

Создадим функцию, которая на основании данных из столбца `'purpose'` сформирует новый столбец `'purpose_category'`, в который войдут следующие категории:
* 'операции с автомобилем'
* 'операции с недвижимостью'
* 'проведение свадьбы'
* 'получение образования'

In [8]:
def purpose_category(item): #создим функцию, которая категоризирует данные purpose
    if 'свадьб' in item:
        return 'проведение свадьбы'
    if 'образован' in item:
        return 'получение образования'
    if 'автомобил' in item:
        return 'операции с автомобилем'
    return 'операции с недвижимостью'
 
data['purpose_category'] = data['purpose'].apply(purpose_category) #создим столбец purpose с категориями
data.head(10) #выведем первые 10 строк датафрейма

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1.0,8437.673028,42.0,0,0,F,сотрудник,0,253875,покупка жилья,B,операции с недвижимостью
1,1.0,4024.803754,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля,C,операции с автомобилем
2,0.0,5623.422610,33.0,1,0,M,сотрудник,0,145885,покупка жилья,C,операции с недвижимостью
3,3.0,4124.747207,32.0,1,0,M,сотрудник,0,267628,дополнительное образование,B,получение образования
4,0.0,340266.072047,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу,C,проведение свадьбы
5,0.0,926.185831,27.0,0,1,M,компаньон,0,255763,покупка жилья,B,операции с недвижимостью
6,0.0,2879.202052,43.0,0,0,F,компаньон,0,240525,операции с жильем,B,операции с недвижимостью
7,0.0,152.779569,50.0,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2.0,6929.865299,35.0,0,1,F,сотрудник,0,95856,на проведение свадьбы,C,проведение свадьбы
9,0.0,2188.756445,41.0,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C,операции с недвижимостью


# 3 Ответы на вопросы.

# Вопрос 1:
* Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [9]:
debt_from_children = pd.DataFrame()
debt_from_children['count_children'] = data.groupby('children')['debt'].count()
debt_from_children['sum_children'] = data.groupby('children')['debt'].sum()
debt_from_children['result_children'] = debt_from_children['sum_children'] / debt_from_children['count_children']*100 
display(debt_from_children)

,count_children,sum_children,result_children
children,,,
0.0,14167,1071,7.559822
1.0,4855,445,9.165808
2.0,2052,194,9.454191
3.0,330,27,8.181818
4.0,41,4,9.756098
5.0,9,0,0.000000


# Вывод: 
С увеличением количества детей мы видим увеличение количества просроченных задолженностей, хотя люди с 3 детьми чаще платят в срок чем люди с 1 ребенком. Данные не однозначные, возможно, нужна большая выборка, чем та, которую мы имеем. Бездетные, как правило реже просрачивают оплату по кредиту, чем люди с детьми.

# Вопрос 2:
* Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [10]:
debt_from_family_status = pd.DataFrame()
debt_from_family_status['sum_family_status'] = data.groupby('family_status_id')['debt'].sum()
debt_from_family_status['count_family_status'] = data.groupby('family_status_id')['debt'].count()
debt_from_family_status['result_family_status'] = debt_from_family_status['sum_family_status'] / debt_from_family_status['count_family_status']*100 
debt_from_family_status.sort_values('result_family_status', ascending = False)

,sum_family_status,count_family_status,result_family_status
family_status_id,,,
4,274,2810,9.750890
1,388,4151,9.347145
0,931,12339,7.545182
3,85,1195,7.112971
2,63,959,6.569343


# Вывод: 
Да, зависимость есть. Люди не в браке и не бывавшие в браке имеют больший процент невозвратов в срок. Но, те кто развелись или овдовели чаще платят в срок, чем люди в браке.

# Вопрос 3:
* Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [11]:
def total_income_category(value):
    if value >= 500000:
        return 10
    else:
        value = value // 50000
        return value
data['total_income_id'] = data['total_income'].apply(total_income_category)
data['total_income_id'].value_counts()
debt_from_total_income = pd.DataFrame()
debt_from_total_income['sum'] = data.groupby('total_income_id')['debt'].sum()
debt_from_total_income['count'] = data.groupby('total_income_id')['debt'].count()
debt_from_total_income['conversion'] = debt_from_total_income['sum'] / debt_from_total_income['count']*100
debt_from_total_income.sort_values('conversion', ascending = False)

,sum,count,conversion
total_income_id,,,
3,368,4118,8.936377
2,661,7807,8.466761
6,51,624,8.173077
1,331,4091,8.090931
4,164,2254,7.275954
7,24,330,7.272727
8,13,196,6.632653
5,88,1330,6.616541
10,14,222,6.306306


# Вывод: 
Исходя из данных таблицы мы видим что есть зависимость от уровня дохода и возврата кредита в срок.

# Вопрос 4:
* Как разные цели кредита влияют на его возврат в срок?

In [12]:
debt_from_purpose_category = pd.DataFrame()
debt_from_purpose_category['sum_purpose_category'] = data.groupby('purpose_category')['debt'].sum()
debt_from_purpose_category['count_purpose_category'] = data.groupby('purpose_category')['debt'].count()
debt_from_purpose_category['result_purpose_category'] = debt_from_purpose_category['sum_purpose_category'] / debt_from_purpose_category['count_purpose_category']*100 
debt_from_purpose_category.sort_values('result_purpose_category', ascending = False)

,sum_purpose_category,count_purpose_category,result_purpose_category
purpose_category,,,
операции с автомобилем,403,4306,9.359034
получение образования,370,4013,9.220035
проведение свадьбы,186,2324,8.003442
операции с недвижимостью,782,10811,7.233373


# Вывод: 
Вот нам и понадобилось разбиение целей кредита по категориям. Как мы видим, среди 4 наших категорий, чаще всего платят в срок с целью кредита недвижимость, чаще просрочка у тех кому нужен кредит на авто и образование. Кредиты на свадьбу просрачивают реже - примерно как и в случае недвижимости.

# Общий вывод:

**В шаге 1.** Мы ознакомились с датафреймом, проверили его на очевидные ошибки и аномалии которые могли быть в названии колонок, содержании или в типе данных. Делается это например для приведения названий столбцов к общему регистру, изменению названий колонок с ошибками ели таковые имеются, приведения типов данных к целочисленному, строковому или иному типу ели это потребуетя. Так же на этом этапе мы можем преблизительно оценить обьем обрабатываемых данных, количество строк, столбцов и пропусков до удаления в них явных дубликатов.

**В шаге 2.1.** Мы приступаем к предварительной обработке данных. Для начала стоит разобраться в пропусках в столбцах, понять возможную причину их появления. Если таковые имеются следует понять какого их количетво и на сколько эти данные важны. Ели их отношение составляет большой процент ко всему датафрейму то стоит задуматься, это может означать что для какой-то категории заемщиков данные не были указанны намеренно в виду их отсутствия. Следовательно, чтобы не терять большой масив заемщиков и применить к ним расчеты, нужно расчитать медианное значение для тех столбцов в которых данные отсутствуют, так сказать установить "среднюю температуру по больнице".

**В шаге 2.2.** Проводим проверку данных на наличие в них аномалий. В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. Например, отрицательное количество дней трудового стажа в столбце 'days_employed'. Для реальных данных это нормально. Обработаем значения в столбцах с аномалиями и опишем возможные причины появления таких данных. После обработки аномалий заполним пропуски в 'days_employed' медианными значениями по этому столбцу.

**В шаге 2.3.** Изменяем тип данныз для столбца 'total_income', так как в этом стобце хранится вещественный тип чисел, а не целочисленный. Для дальнейших расчетов нам будет удобнее пользоваться целыми числами.

**В шаге 2.4.** Удаляем дубликаты. В данных присутствуют строки-дубликаты, удалим их. Также обработаем неявные дубликаты. Например, в столбце 'education' есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к одному регистру. 

**В шаге 2.5.** Формируем дополнительные датафреймы-словари, декомпозиция исходного датафрейма.
Создадим два новых датафрейма, в которых: каждому уникальному значению из 'education' соответствует уникальное значение 'education_id' — в первом; каждому уникальному значению из 'family_status' соответствует уникальное значение 'family_status_id' — во втором.
Удалим дубликаты из словарей методом .drop_duplicates()
Удалим из исходного датафрейма столбцы 'education' и 'family_status', оставив только их идентификаторы: 'education_id' и 'family_status_id'.

**В шаге 2.6.** Категоризируем доход, создаем новый столбец в котором лицам с определенным типом дохода будет присвоенна своя категория, например: лицам чей доход меньше или равен 30000 т., р., будет присваиваться категория 'Е'.

**В шаге 2.7.** Категоризируем цели кредита. Создадим функцию, которая на основании данных из столбца 'purpose' сформирует новый столбец 'purpose_category', в который войдут следующие категории: 'операции с автомобилем', 'операции с недвижимостью', 'проведение свадьбы', 'получение образования'.

Общий вывод - семейный статус влияет на вероятность платежей по кредиту в срок - люди, бывший в браке чаще плятят в срок чем те, кто не бывали в браке, причем разведенные и овдовевшие платят в срок чаще, чем люди в браке. Чем больше детей, тем чаще платят не в срок, бездетные реже просрачивают оплату.